# Inference

## Dependencies and imports

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## Load fintuned model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("VerbACxSS/sempl-it-mt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("VerbACxSS/sempl-it-mt5-small")
model.to('cuda')

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

## Load datasets

In [ ]:
train_df = pd.read_csv('../dataset/train.tsv', sep='\t', encoding='utf-8')
test_df = pd.read_csv('../dataset/test.tsv', sep='\t', encoding='utf-8')
val_df = pd.read_csv('../dataset/val.tsv', sep='\t', encoding='utf-8')

## Utilities

In [ ]:
PREFIX = "semplifica: "
TOKENIZER_MAX_LENGHT = 1024

def predict(samples_array, batch_size=16):
  prompts = [PREFIX + s for s in samples_array]
  batched_prompt = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

  outputs = []
  for batch in tqdm(batched_prompt):
    x = tokenizer(batch, max_length=TOKENIZER_MAX_LENGHT, truncation=True, padding=True, return_tensors='pt').input_ids.to('cuda')
    y =  model.generate(x, max_new_tokens=TOKENIZER_MAX_LENGHT)
    decoded = tokenizer.batch_decode(y, skip_special_tokens=True, clean_up_tokenization_spaces=True, max_length=TOKENIZER_MAX_LENGHT, truncation=True)
    outputs.extend(decoded)
  return outputs

## Random predictions

In [ ]:
for s in test_df.sample(10).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\noracle: ", s['oracle_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:01<00:00,  1.37s/it]



original:  Il Servizio Residenziale Terapeutico Riabilitativo è rivolto a pazienti con media-alta motivazione al cambiamento e risorse adeguate a intraprendere percorsi di comprensione della patologia e di modificazione dei patterns emotivo/cognitivo/comportamentali.

oracle:  Il Servizio Residenziale Terapeutico Riabilitativo è pensato per persone che sono abbastanza motivate a cambiare e hanno le risorse necessarie per capire la loro malattia e migliorare il modo in cui pensano, si sentono e si comportano.

model:  Il Servizio Residenziale Terapeutico Riabilitativo è pensato per pazienti con media-alta motivazione a cambiare e a prendere decisioni su come cambiare le loro emozioni e cambiare i motivi emotivi e comportamentali.
----------------


100%|██████████| 1/1 [00:00<00:00,  1.18it/s]



original:  monitorare i consumi e la spesa dei DM e IVD e, in funzione delle priorità regionali, effettuare valutazioni per l'uso appropriato e razionale delle risorse; 

oracle:  Controllare quanto vengono utilizzati e quanto si spende per i dispositivi medici e i dispositivi diagnostici in vitro, e valutare se vengono usati in modo corretto e conveniente in base alle esigenze della regione.

model:  Controllare i consumi e la spesa dei medici di base e delle scuole e dei servizi sanitari regionali. In base alle priorità regionali, fare valutazioni per usare adatti e razionali le risorse.
----------------


100%|██████████| 1/1 [00:01<00:00,  1.50s/it]


...


original:  Dall'analisi dei flussi di mobilità passiva della regione Molise, come rappresentato nella tabella seguente, è emerso quanto segue: nell'anno 2018, si sono registrati 10.270 casi, per un valore pari a 38,634 milioni di euro. Il 55% dei casi (59% del valore) è stato erogato da strutture private accreditate. Nel 2019 si sono registrati 10.119 casi, per un valore pari a 38,367 milioni di euro, con il 56% dei casi (61% del valore) erogato da strutture private accreditate. Nel 2020 si sono registrati 7.067 casi, per un valore pari a 27.835 milioni di euro, con il 57% dei casi (62% dei valori) erogato da strutture private accreditate. Le Regioni che assorbono la maggiore mobilità di casi nell'anno 2020 sono risultate essere: l'Abruzzo, il Lazio, la Puglia e la Campania. Tuttavia, i valori di mobilità passiva verso le regioni Emilia-Romagna e Lombardia, per l'anno 2020, risultano più elevati del valore della Campania, come anche negli anni 2018 e 2019.

oracle:  Dall'analisi dei t

100%|██████████| 1/1 [00:00<00:00,  2.11it/s]



original:  l'ampliamento degli orari di visita e di assistenza per i familiari.

oracle:  I tempi in cui i familiari possono visitare e aiutare i loro cari saranno più lunghi.

model:  L'aumento degli orari di visita e assistenza per le famiglie è importante per i loro familiari.
----------------


100%|██████████| 1/1 [00:02<00:00,  2.29s/it]


original:  Infine, un ambito strategico distinto dai precedenti e rispetto al quale è necessario intervenire, specie in prospettiva preventiva, riguarda l'individuazione precoce dei minori con problematiche comportamentali, in considerazione, da un lato, dell'incremento di tali situazioni di fragilità e disagio, dall'altro, dell'efficacia di programmi riabilitativi avviati già in età infantile nella modifica del decorso dei problemi di comportamento, con guadagni in termini di minori limitazioni e minor fabbisogno di sostegni in età adulta. A tal fine è prioritario prevede la sperimentazione di metodologie e strumenti multidimensionali per lo screening precoce delle predette problematiche comportamentali, con l'implementazione di tecniche riabilitative specifiche che contemplino anche il coinvolgimento diretto del mondo della scuola e della famiglia attraverso appositi percorsi formativi e informativi.

oracle:  Un'altra area importante su cui concentrarsi è l'individuazione precoce d

## Run all predictions

In [ ]:
train_df['model_text'] = predict(train_df['original_text'].tolist(), batch_size=16)
test_df['model_text'] = predict(test_df['original_text'].tolist(), batch_size=16)
val_df['model_text'] = predict(val_df['original_text'].tolist(), batch_size=16)

100%|██████████| 115/115 [06:16<00:00,  3.27s/it]


## Save simplified datasets

In [ ]:
train_df.to_csv('./train_simplified.tsv', index=False, sep='\t')
test_df.to_csv('./test_simplified.tsv', index=False, sep='\t')
val_df.to_csv('./val_simplified.tsv', index=False, sep='\t')